In [1]:
%reload_ext autoreload
%autoreload 2

import os
import glob
from pathlib import Path

import numpy as np
import pandas as pd

from tqdm import tqdm
from hbmep.utils import timing
import mat73
import tomllib


In [2]:
MUSCLES = ["ADM", "APB", "Biceps", "ECR", "FCR", "Triceps"]
PKPK_MUSCLES = ["PKPK_" + m for m in MUSCLES]
AUC_MUSCLES = ["AUC_" + m for m in MUSCLES]

@timing
def load_tms_data(
    dir: Path,
    subjects: list[str]
):
    df = None

    for subject in tqdm(subjects):
        subdir = os.path.join(dir, subject)

        fpath = glob.glob(f"{subdir}/*REC_table.csv")[0]
        temp_df = pd.read_csv(fpath)

        fpath = glob.glob(f"{subdir}/*ep_matrix.mat")[0]
        data_dict = mat73.loadmat(fpath)

        temp_mat = data_dict["ep_sliced"]

        fpath = glob.glob(f"{subdir}/*cfg_proc.toml")[0]
        with open(fpath, "rb") as f:
            cfg_proc = tomllib.load(f)

        temp_df["participant"] = subject

        # Rename columns to actual muscle names
        muscles = cfg_proc["st"]["channel"]
        pkpk_muscles_map = {
            f"pkpk_{i + 1}": "PKPK_" + m for i, m in enumerate(muscles)
        }
        auc_muscles_map = {
            f"auc_{i + 1}": "AUC_" + m for i, m in enumerate(muscles)
        }
        temp_df = temp_df.rename(columns=pkpk_muscles_map).copy()
        temp_df = temp_df.rename(columns=auc_muscles_map).copy()

        # Reorder MEP matrix
        temp_mat = temp_mat[..., np.argsort(muscles)]

        assert temp_df["target_muscle"].unique().shape[0] == 1
        side = temp_df["target_muscle"].unique()[0][0]

        pkpk_side_muscles = ["PKPK_" + side + muscle for muscle in MUSCLES]
        temp_df[PKPK_MUSCLES] = temp_df[pkpk_side_muscles]

        auc_side_muscles = ["AUC_" + side + muscle for muscle in MUSCLES]
        temp_df[AUC_MUSCLES] = temp_df[auc_side_muscles]

        side_muscles = [side + muscle for muscle in MUSCLES]

        ind = [i for i, m in enumerate(sorted(muscles)) if m in side_muscles]
        temp_mat = temp_mat[..., ind]

        if df is None:
            df = temp_df.copy()
            mat = temp_mat

            # time = data_dict["t_sliced"]
            # auc_window = cfg_proc["auc"]["t_slice_minmax"]
            muscles_sorted = sorted(muscles)

            assert len(set(muscles_sorted)) == len(muscles_sorted)
            continue

        # assert (data_dict["t_sliced"] == time).all()
        # assert cfg_proc["auc"]["t_slice_minmax"] == auc_window
        assert set(muscles) == set(muscles_sorted)

        df = pd.concat([df, temp_df], ignore_index=True).reset_index(drop=True).copy()
        mat = np.vstack((mat, temp_mat))

    # # Rename df response columns to auc_i
    # pkpk_muscles_map = {
    #     m: f"pkpk_{i + 1}" for i, m in enumerate(muscles_sorted)
    # }
    # auc_muscles_map = {
    #     m: f"auc_{i + 1}" for i, m in enumerate(muscles_sorted)
    # }
    # df = df.rename(columns=muscles_map).reset_index(drop=True).copy()

    # muscles_map = {
    #     v: u for u, v in muscles_map.items()
    # }
    return df, mat, pkpk_muscles_map, auc_muscles_map

In [3]:
# dir = "/mount/hdd1/human_non-inv/proc"
dir = "/mount/hdd1/human_non-inv/proc-2023-10-27"
# subjects = ["SCA01", "SCA02", "SCA04", "SCA05", "SCA06", "SCA07", "SCA09", "SCA10", "SCA11", "SCS01", "SCS02", "SCS03", "SCS04"]
subjects = ["SCA01", "SCA02", "SCA03", "SCA04", "SCA05", "SCA06", "SCA07", "SCA09", "SCA10", "SCA11", "SCS01", "SCS02", "SCS03", "SCS04", "SCS05", "SCS06"]

df, mat, pkpk_muscles_map, auc_muscles_map = load_tms_data(dir=dir, subjects=subjects)

df.shape

  0%|                                                  | 0/16 [00:00<?, ?it/s]

100%|█████████████████████████████████████████| 16/16 [00:02<00:00,  6.60it/s]
2023-11-08 11:49:19,459 - hbmep.utils.utils - INFO - func:load_tms_data took: 2.43 sec


(1112, 127)

In [4]:
mat.shape

(1112, 1501, 6)

In [5]:
target_df = df \
    .groupby(by=["participant"], as_index=False) \
    .agg({"target_muscle": np.unique}) \
    .explode(column="target_muscle") \
    .reset_index(drop=True) \
    .copy()

target_df

,participant,target_muscle
0,SCA01,LAPB
1,SCA02,RAPB
2,SCA03,RAPB
3,SCA04,RAPB
4,SCA05,RAPB
5,SCA06,RAPB
6,SCA07,RAPB
7,SCA09,RAPB
8,SCA10,RAPB
9,SCA11,RAPB


In [6]:
for subject in df.participant.unique():
    ind = df.participant.isin([subject])
    temp_df = df[ind].reset_index(drop=True).copy()

    side = temp_df.target_muscle.unique()[0][0]
    pkpk_sided_muscles = ["PKPK_" + side + m for m in MUSCLES]
    auc_sided_muscles = ["AUC_" + side + m for m in MUSCLES]

    assert (temp_df[PKPK_MUSCLES].to_numpy() == temp_df[pkpk_sided_muscles].to_numpy()).all()
    assert (temp_df[AUC_MUSCLES].to_numpy() == temp_df[auc_sided_muscles].to_numpy()).all()


In [7]:
dst = "/home/vishu/data/hbmep-processed/human/tms"
df.to_csv(os.path.join(dst, "data_pkpk_auc_proc-2023-10-27.csv"), index=False)

np.save(os.path.join(dst, "mep_matrix_pkpk_auc_proc-2023-10-27.npy"), mat)
# np.save(os.path.join(dst, "auc_window.npy"), np.array(auc_window))


In [8]:
import json

f = open(os.path.join(dst, "pkpk_muscles_map.json"), "w")
f.write(json.dumps(pkpk_muscles_map))
f.close;

f = open(os.path.join(dst, "auc_muscles_map.json"), "w")
f.write(json.dumps(auc_muscles_map))
f.close;


In [9]:
pkpk_muscles_map

{'pkpk_1': 'PKPK_LBiceps',
 'pkpk_2': 'PKPK_RBiceps',
 'pkpk_3': 'PKPK_LTriceps',
 'pkpk_4': 'PKPK_RTriceps',
 'pkpk_5': 'PKPK_LFCR',
 'pkpk_6': 'PKPK_RFCR',
 'pkpk_7': 'PKPK_LECR',
 'pkpk_8': 'PKPK_RECR',
 'pkpk_9': 'PKPK_LAPB',
 'pkpk_10': 'PKPK_RAPB',
 'pkpk_11': 'PKPK_LADM',
 'pkpk_12': 'PKPK_RADM',
 'pkpk_13': 'PKPK_LFDI',
 'pkpk_14': 'PKPK_RFDI',
 'pkpk_15': 'PKPK_LTA',
 'pkpk_16': 'PKPK_RTA',
 'pkpk_17': 'PKPK_trigger_tscs',
 'pkpk_18': 'PKPK_trigger_peripheral',
 'pkpk_19': 'PKPK_trigger_tms'}

In [10]:
auc_muscles_map

{'LADM': 'auc_1',
 'LAPB': 'auc_2',
 'LBiceps': 'auc_3',
 'LECR': 'auc_4',
 'LFCR': 'auc_5',
 'LFDI': 'auc_6',
 'LTA': 'auc_7',
 'LTriceps': 'auc_8',
 'RADM': 'auc_9',
 'RAPB': 'auc_10',
 'RBiceps': 'auc_11',
 'RECR': 'auc_12',
 'RFCR': 'auc_13',
 'RFDI': 'auc_14',
 'RTA': 'auc_15',
 'RTriceps': 'auc_16',
 'trigger_peripheral': 'auc_17',
 'trigger_tms': 'auc_18',
 'trigger_tscs': 'auc_19'}

In [11]:
fpath = os.path.join(dir, "SCA01", "SCA01_V1T0_TMS_REC_ep_matrix.mat")
data_dict = mat73.loadmat(fpath)

t = data_dict["t_sliced"]
print(t.min(), t.max())

-0.15 0.15


In [12]:
df.shape

(568, 114)

In [13]:
mat.shape


(568, 1501, 0)

In [14]:
sorted(MUSCLES) == MUSCLES


True